In [16]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import to_categorical
%matplotlib inline

In [3]:
df = pd.read_csv('./data/3227set.csv', index_col = 0)

In [4]:
df['mid1'] = (df['Ask1'] + df['Bid1']) / 2

In [5]:
df = df.reset_index(drop = True)

In [6]:
df = df.drop([0, 253059])
df = df.reset_index(drop = True)

In [7]:
df = df.drop(['next_mid1'], axis = 1)

In [25]:
sum(df['Ask2'] == 0)

44

In [26]:
sum(df['Ask3'] == 0)

510

In [27]:
sum(df['Ask4'] == 0)

1273

In [28]:
sum(df['Ask5'] == 0)

1617

In [17]:
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253053,212,130.0,130.5,23,142,129.5,131.0,24,316,129.0,...,-2.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,130.25
253054,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
253055,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
253056,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [18]:
#normalize
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm
#rolling windows
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["mid1"]))
    return np.array(X_train), np.array(Y_train)
#LSTM
def lstm_stock_model(shape):
    model = Sequential()
    model.add(LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    model.add(Dense(5,activation='linear'))
    model.add(Dense(1,activation='linear'))
    model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error'])
    model.summary()
    return model


In [26]:
df.shape[0]*0.3

75917.4

In [27]:
test = df[-round(df.shape[0]*0.3):]
test

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
177141,59,123.0,123.5,50,140,122.5,124.0,157,125,122.0,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,123.25
177142,60,123.0,123.5,50,140,122.5,124.0,157,125,122.0,...,0.0,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,123.25
177143,60,123.0,123.5,50,140,122.5,124.0,158,125,122.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.25
177144,59,123.0,123.5,51,140,122.5,124.0,158,125,122.0,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,123.25
177145,60,123.0,123.5,51,140,122.5,124.0,158,125,122.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,123.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253053,212,130.0,130.5,23,142,129.5,131.0,24,316,129.0,...,-2.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,130.25
253054,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
253055,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
253056,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [30]:
test = test.reset_index()
test

,index,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,177141,59,123.0,123.5,50,140,122.5,124.0,157,125,...,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,123.25
1,177142,60,123.0,123.5,50,140,122.5,124.0,157,125,...,0.0,1.0,-2.0,0.0,1.0,0.0,0.0,0.0,0.0,123.25
2,177143,60,123.0,123.5,50,140,122.5,124.0,158,125,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.25
3,177144,59,123.0,123.5,51,140,122.5,124.0,158,125,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,123.25
4,177145,60,123.0,123.5,51,140,122.5,124.0,158,125,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,123.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75912,253053,212,130.0,130.5,23,142,129.5,131.0,24,316,...,-2.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,130.25
75913,253054,212,130.0,130.5,22,142,129.5,131.0,24,316,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
75914,253055,213,130.0,130.5,22,142,129.5,131.0,24,316,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
75915,253056,213,130.0,130.5,15,142,129.5,131.0,25,312,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [29]:
train = df[:round(df.shape[0]*0.7)]
train

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177136,58,123.0,123.5,50,140,122.5,124.0,155,123,122.0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,123.25
177137,58,123.0,123.5,48,140,122.5,124.0,155,123,122.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,123.25
177138,58,123.0,123.5,50,140,122.5,124.0,156,123,122.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.25
177139,58,123.0,123.5,50,140,122.5,124.0,156,123,122.0,...,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,123.25


In [ ]:
#720's 5sec -> 1hours
X_train, Y_train = train_windows(train, 720, 1)
X_test, Y_test = train_windows(test, 720, 1)

In [32]:
model = lstm_stock_model(X_train.shape)
callback = EarlyStopping(monitor="mean_absolute_error", patience=10, verbose=1, mode="auto")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 256)           329728    
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             257       
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 855,358
Trainable params: 855,358
Non-trainable params: 0
________________________________________________

In [33]:
history = model.fit(X_train, Y_train, epochs=1000, batch_size=5, validation_split=0.1, callbacks=[callback],shuffle=True)

Train on 159417 samples, validate on 17713 samples
Epoch 1/1000
 89805/159417 [===============>..............] - ETA: 10:56 - loss: 3.9627 - mean_absolute_error: 3.9627

KeyboardInterrupt: 